In [1]:
!pip install opencv-python
!pip install numpy
!pip install matplotlib

In [8]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 11 14:34:34 2017

@author: ukalwa
"""
from __future__ import print_function

import math

import cv2
import numpy as np
import pickle


win_name = "VSS Trackbars"

# List of sigs
listOfHSV_Sigs = {}
NUM_OF_HSV_SIGS = 8

DEFAULT_SIG = [0, 0, 0, 0, 0, 0, 0]

def saveSignatures():



    try:
        fileObj = open('vss_signatures.pk1', "wb")
    except:
        print("Failed To Save!")
        return 1

    pickle.dump(listOfHSV_Sigs, fileObj)
    print("Saved!")
          
    fileObj.close()

    return 0


def loadSignatures():

    global listOfHSV_Sigs

    try:

        fileObj = open('vss_signatures.pk1', "rb")

        signatures = pickle.load(fileObj)

        listOfHSV_Sigs = signatures

        print(listOfHSV_Sigs)

        print("loaded")

        fileObj.close()

    except:
        print("Failed To Load!")
        return 1




def dummy(x):
    return x


def onmouse(event, x, y, flags, params):
    global point
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)
        print(x, y)


point = None
prev_point = None

def boxColor(index):

    match index:
        case 0:
            return (255, 100, 100)
        case 1:
            return (100, 100, 255)
        case 2:
            return (255, 100, 255)
        case 3:
            return (100, 255, 100)
        case 4:
            return (255, 255, 100)
        case 5:
            return (100, 255, 255)
        case 6:
            return (255, 255, 255)


def detect_object(image, res, threshold, currentSig, rectColor, objectArray):
    res = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    res = cv2.blur(res, (5,5))

    hul = listOfHSV_Sigs.get(currentSig)[0]
    huh = listOfHSV_Sigs.get(currentSig)[1]
    sal = listOfHSV_Sigs.get(currentSig)[2]
    sah = listOfHSV_Sigs.get(currentSig)[3]
    val = listOfHSV_Sigs.get(currentSig)[4]
    vah = listOfHSV_Sigs.get(currentSig)[5]
    run = listOfHSV_Sigs.get(currentSig)[6]

    if run == 1:

        hsvRange = (hul, huh, sal, sah, val, vah)

        cnts, hierarchy = cv2.findContours(res, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for cnt in np.arange(len(cnts)):
            rect = cv2.boundingRect(cnts[cnt])
            x, y, w, h = rect
            if w*h >= threshold*300:
                cv2.rectangle(res, (x, y), (x + w, y + h), (255, 255, 255), -1)


        cnts, hierarchy = cv2.findContours(res, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        mask2 = np.zeros(res.shape)
        cv2.drawContours(mask2, cnts, -1, (255, 255, 255), -1)

        for cnt in np.arange(len(cnts)):
            rect = cv2.boundingRect(cnts[cnt])
            x, y, w, h = rect
            objectArray.append([rect, hsvRange])
            #print(objBoxArray)
            if w*h >= threshold*100:
                cv2.rectangle(image, (x, y), (x + w, y + h), rectColor, 2)
                cv2.putText(image, str(currentSig), (x + math.ceil(w/4),y + math.ceil(h/4)), cv2.FONT_HERSHEY_SIMPLEX,
                            .5, rectColor, 1, cv2.LINE_AA)
        return len(cnts)

    else:
        print("Signature not defined.")
        return 0

def setup(savedSignatures = {}):

    # easy assigments
    cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
    cv2.setMouseCallback(win_name, onmouse)

    cv2.createTrackbar('H Low', win_name, 0, 255, dummy)
    cv2.createTrackbar('H High', win_name, 0, 255, dummy)
    cv2.createTrackbar('S Low', win_name, 0, 255, dummy)
    cv2.createTrackbar('S High', win_name, 0, 255, dummy)
    cv2.createTrackbar('V Low', win_name, 0, 255, dummy)
    cv2.createTrackbar('V High', win_name, 0, 255, dummy)
    cv2.createTrackbar('Object Sensitivity', win_name, 0, 10, dummy)
    cv2.createTrackbar('Signature', win_name, 0, NUM_OF_HSV_SIGS - 2, dummy)

    for i in range(0, NUM_OF_HSV_SIGS - 1):
        listOfHSV_Sigs[i] = DEFAULT_SIG

    loadSignatures()

    #print(listOfHSV_Sigs)


    cv2.setTrackbarPos('H Low', win_name, listOfHSV_Sigs.get(0)[0])
    cv2.setTrackbarPos('H High', win_name, listOfHSV_Sigs.get(0)[1])
    cv2.setTrackbarPos('S Low', win_name, listOfHSV_Sigs.get(0)[2])
    cv2.setTrackbarPos('S High', win_name, listOfHSV_Sigs.get(0)[3])
    cv2.setTrackbarPos('V Low', win_name, listOfHSV_Sigs.get(0)[4])
    cv2.setTrackbarPos('V High', win_name, listOfHSV_Sigs.get(0)[5])

    print("set trackbars")

    print(listOfHSV_Sigs)

def stepVSS(webcamFeed):

    global lastSigAccessed
    global listOfHSV_Sigs

    try:
        lastSigAccessed
    except:
        lastSigAccessed = 0

    ret, image = webcamFeed.read()
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Gets the current signature
    currentSig = cv2.getTrackbarPos('Signature', win_name)



    # read trackbar positions for all
    hul = cv2.getTrackbarPos('H Low', win_name)
    huh = cv2.getTrackbarPos('H High', win_name)
    sal = cv2.getTrackbarPos('S Low', win_name)
    sah = cv2.getTrackbarPos('S High', win_name)
    val = cv2.getTrackbarPos('V Low', win_name)
    vah = cv2.getTrackbarPos('V High', win_name)
    obj_threshold = cv2.getTrackbarPos('Object Sensitivity', win_name)



    if currentSig != lastSigAccessed:
        # Pulls data from hashmap so the bars go to their saved position
        cv2.setTrackbarPos('H Low', win_name, listOfHSV_Sigs.get(currentSig)[0])
        cv2.setTrackbarPos('H High', win_name, listOfHSV_Sigs.get(currentSig)[1])
        cv2.setTrackbarPos('S Low', win_name, listOfHSV_Sigs.get(currentSig)[2])
        cv2.setTrackbarPos('S High', win_name, listOfHSV_Sigs.get(currentSig)[3])
        cv2.setTrackbarPos('V Low', win_name, listOfHSV_Sigs.get(currentSig)[4])
        cv2.setTrackbarPos('V High', win_name, listOfHSV_Sigs.get(currentSig)[5])




    run = 1

    # Sets the signature that is active to be whatever the current trackbars values are.
    listOfHSV_Sigs[currentSig] = [hul, huh, sal, sah, val, vah, run]

    print(listOfHSV_Sigs)

    lastSigAccessed = currentSig

    objectList = []

    for sig in range(0, NUM_OF_HSV_SIGS - 1):

        # make arrays for final values

        # H, S, V
        hsv_low = np.array([listOfHSV_Sigs.get(sig)[0], listOfHSV_Sigs.get(sig)[2], listOfHSV_Sigs.get(sig)[4]])
        hsv_high = np.array([listOfHSV_Sigs.get(sig)[1], listOfHSV_Sigs.get(sig)[3], listOfHSV_Sigs.get(sig)[5]])

        # apply the range on a mask
        mask = cv2.inRange(hsv_image, hsv_low, hsv_high)
        res = cv2.bitwise_and(image, image, mask=mask)

        count = detect_object(image, res, obj_threshold, sig,boxColor(sig), objectList)

    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(image, 'Press q to quit,', (10, 30), font, .5, (204, 255, 255), 1)
    cv2.putText(image, 'Press z to pause,', (10, 50), font, .5, (204, 255, 255), 1)
    cv2.putText(image, 'Press s to save sigs,', (200, 30), font, .5, (204, 255, 255), 1)
    cv2.putText(image, 'Press x to reset sigs', (200, 50), font, .5, (204, 255, 255), 1)

    cv2.imshow("VSS Display", image)


    key = cv2.waitKey(5) & 0xFF
    if key == ord('q'):
        webcamFeed.release()
        cv2.destroyAllWindows()
    if key == ord('s'):
        saveSignatures()
    if key == ord('x'):
        for i in range(0, NUM_OF_HSV_SIGS - 1):
            listOfHSV_Sigs[i] = DEFAULT_SIG
        # Pulls data from hashmap so the bars go to their saved position
        cv2.setTrackbarPos('H Low', win_name, listOfHSV_Sigs.get(currentSig)[0])
        cv2.setTrackbarPos('H High', win_name, listOfHSV_Sigs.get(currentSig)[1])
        cv2.setTrackbarPos('S Low', win_name, listOfHSV_Sigs.get(currentSig)[2])
        cv2.setTrackbarPos('S High', win_name, listOfHSV_Sigs.get(currentSig)[3])
        cv2.setTrackbarPos('V Low', win_name, listOfHSV_Sigs.get(currentSig)[4])
        cv2.setTrackbarPos('V High', win_name, listOfHSV_Sigs.get(currentSig)[5])
    if key == ord('z'):
        while True:
            loopkey = cv2.waitKey(100) & 0xFF
            if loopkey == ord('z'):
                break
            if loopkey == ord('q'):
                webcamFeed.release()
                cv2.destroyAllWindows()
            if loopkey == ord('s'):
                saveSignatures()
            if key == ord('x'):
                for i in range(0, NUM_OF_HSV_SIGS - 1):
                    listOfHSV_Sigs[i] = DEFAULT_SIG
                # Pulls data from hashmap so the bars go to their saved position
                cv2.setTrackbarPos('H Low', win_name, listOfHSV_Sigs.get(currentSig)[0])
                cv2.setTrackbarPos('H High', win_name, listOfHSV_Sigs.get(currentSig)[1])
                cv2.setTrackbarPos('S Low', win_name, listOfHSV_Sigs.get(currentSig)[2])
                cv2.setTrackbarPos('S High', win_name, listOfHSV_Sigs.get(currentSig)[3])
                cv2.setTrackbarPos('V Low', win_name, listOfHSV_Sigs.get(currentSig)[4])
                cv2.setTrackbarPos('V High', win_name, listOfHSV_Sigs.get(currentSig)[5])
                    

    return objectList



In [9]:
import cv2

def getBySize(objArray, comparitive_size = 0):

    if len(objArray) == 0:
        return None
    if len(objArray) == 1:
        #print(objArray[0][0][0])
        return objArray[0][0][0]

    #print(objArray)

    if comparitive_size > len(objArray) - 1:
        comparitive_size = len(objArray) - 1

    # Insertion Sort

    # Starts from the second element
    for i in range(1, len(objArray)):

        rectToMove = objArray[i]

        # i is only used here
        rectToLeftIndex = i - 1

        # While the RTLI var is greater than 0 and won't throw an access error and
        # while the RTLI area > rectToMove area, which checks if the box to the left of
        # the RTM is greater than RTM. If so, it moves it over one. Keeps doing this until
        # RTLI is less than RTM, which means it's sorted.
        while (rectToLeftIndex >= 0 and objArray[rectToLeftIndex][0][2]*objArray[rectToLeftIndex][0][3] >
               rectToMove[0][2]*rectToMove[0][3] ):

            # Shifts rectangle to the left
            objArray[rectToLeftIndex + 1] = objArray[rectToLeftIndex]

            # Shifts index to the left
            rectToLeftIndex -= 1

        # Shifts the rectangle to the right location
        objArray[rectToLeftIndex + 1] = objArray[rectToLeftIndex]

    # Makes a new array the inverse of objArray, so biggest is 0. I don't get the syntax, but it's whatever.
    newObjArray = objArray[::-1]

    #print(newObjArray[0][0][comparitive_size])
    return newObjArray[comparitive_size][0]

def getByHSV_Sig(objArray, HSV_sig_param):

    returnObjArray = []

    hsv_sig = listOfHSV_Sigs[HSV_sig_param]

    if len(objArray) == 0:
        return None

    #print(len(objArray))

    for i in range(0, len(objArray)-1):
        #print("HSV For Loop: ", i)
        #print("   ", objArray[i][1], "    ", hsv_sig)

        if (objArray[i][1][0] == hsv_sig[0] and
            objArray[i][1][1] == hsv_sig[1] and
            objArray[i][1][2] == hsv_sig[2] and
            objArray[i][1][3] == hsv_sig[3] and
            objArray[i][1][4] == hsv_sig[4] and
            objArray[i][1][5] == hsv_sig[5]):

            #print("Found matching sig")
            if len(objArray) == 1:
                return objArray[i][0]
            returnObjArray.append(objArray[i][0])


    if len(returnObjArray) == 0:
        return None

    # Insertion Sort

    #print ("Insertion Sort")

    # Starts from the second element
    for i in range(1, len(returnObjArray)):

        #print("Iteration: ", i)
        rectToMove = returnObjArray[i]

        # i is only used here
        rectToLeftIndex = i - 1

        # While the RTLI var is greater than 0 and won't throw an access error and
        # while the RTLI area > rectToMove area, which checks if the box to the left of
        # the RTM is greater than RTM. If so, it moves it over one. Keeps doing this until
        # RTLI is less than RTM, which means it's sorted.
        while (rectToLeftIndex >= 0 and returnObjArray[rectToLeftIndex][2] * returnObjArray[rectToLeftIndex][3] >
               rectToMove[2] * rectToMove[3]):
            # Shifts rectangle to the left
            returnObjArray[rectToLeftIndex + 1] = returnObjArray[rectToLeftIndex]

            # Shifts index to the left
            rectToLeftIndex -= 1

        # Shifts the rectangle to the right location
        returnObjArray[rectToLeftIndex + 1] = returnObjArray[rectToLeftIndex]

        #print(returnObjArray)

    # Makes a new array the inverse of objArray, so biggest is 0. I don't get the syntax, but it's whatever.
    returnObjArray = returnObjArray[::-1]

    #print(returnObjArray)

    return returnObjArray

def getObjCnt(objArray, HSV_sig = None):
    return len(objArray)

def getCameraWidth(camera):
    return camera.get(cv2.CAP_PROP_FRAME_WIDTH)

def getCameraHeight(camera):
    return camera.get(cv2.CAP_PROP_FRAME_HEIGHT)







In [ ]:
# -*- coding: utf-8 -*-
"""
Program heavily influenced by Github user Ukalwa who made a blob detection algorithim for his game.

Link to project I used code from:
https://github.com/ukalwa/blob_color_detection/tree/master

"""

import cv2

vid = cv2.VideoCapture(0)
bool_read, image = vid.read()
frame = 0
if bool_read:

    setup()

    while True:

        objArray = stepVSS(vid)

        HSVBox = getByHSV_Sig(objArray, 0)

        #print(HSVBox)

        if HSVBox is not None:
            x, y, width, height = HSVBox[0]
            #print(width * height)





cv2.destroyAllWindows()
vid.release()




{0: [10, 43, 153, 255, 153, 255, 1], 1: [0, 0, 0, 0, 0, 0, 0], 2: [0, 0, 0, 0, 0, 0, 0], 3: [0, 0, 0, 0, 0, 0, 0], 4: [0, 0, 0, 0, 0, 0, 0], 5: [0, 0, 0, 0, 0, 0, 0], 6: [0, 0, 0, 0, 0, 0, 0]}
loaded
set trackbars
{0: [10, 43, 153, 255, 153, 255, 1], 1: [0, 0, 0, 0, 0, 0, 0], 2: [0, 0, 0, 0, 0, 0, 0], 3: [0, 0, 0, 0, 0, 0, 0], 4: [0, 0, 0, 0, 0, 0, 0], 5: [0, 0, 0, 0, 0, 0, 0], 6: [0, 0, 0, 0, 0, 0, 0]}
{0: [10, 43, 153, 255, 153, 255, 1], 1: [0, 0, 0, 0, 0, 0, 0], 2: [0, 0, 0, 0, 0, 0, 0], 3: [0, 0, 0, 0, 0, 0, 0], 4: [0, 0, 0, 0, 0, 0, 0], 5: [0, 0, 0, 0, 0, 0, 0], 6: [0, 0, 0, 0, 0, 0, 0]}
Signature not defined.
Signature not defined.
Signature not defined.
Signature not defined.
Signature not defined.
Signature not defined.
{0: [10, 43, 153, 255, 153, 255, 1], 1: [0, 0, 0, 0, 0, 0, 0], 2: [0, 0, 0, 0, 0, 0, 0], 3: [0, 0, 0, 0, 0, 0, 0], 4: [0, 0, 0, 0, 0, 0, 0], 5: [0, 0, 0, 0, 0, 0, 0], 6: [0, 0, 0, 0, 0, 0, 0]}
Signature not defined.
Signature not defined.
Signature not defined.
